# Rapid-Aid Project ML Core : 

In [1]:
#Import All The Libraries 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_absolute_error, classification_report

#Remove Warnings
import warnings
warnings.filterwarnings("ignore")

#Import The Dataset
df = pd.read_csv("emergency_ml_dataset_1000.csv")

#Data Labelling
categorical_cols = [
    "emergency_type",
    "time_of_day",
    "weather_condition",
    "severity_class",
    "outcome"
]
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

#Data Preprocessing
X = df[
    [
        "emergency_type",
        "severity_reported",
        "distance_km",
        "traffic_level",
        "time_of_day",
        "people_affected",
        "ambulance_available",
        "hospital_icu_available",
        "weather_condition",
        "area_risk_score"
    ]
]
y_severity = df["severity_class"]
y_response_time = df["actual_response_time"]
y_outcome = df["outcome"]

#Data Splitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_sev_train, y_sev_test = train_test_split(X_scaled, y_severity, test_size=0.2, random_state=42)
_, _, y_time_train, y_time_test = train_test_split(X_scaled, y_response_time, test_size=0.2, random_state=42)
_, _, y_out_train, y_out_test = train_test_split(X_scaled, y_outcome, test_size=0.2, random_state=42)

#RandomForest Model Classification - (Severity)
severity_model = RandomForestClassifier(n_estimators=100,random_state=42)
severity_model.fit(X_train, y_sev_train)
sev_pred = severity_model.predict(X_test)
print("Severity Model Accuracy:", accuracy_score(y_sev_test, sev_pred))
print(classification_report(y_sev_test, sev_pred))

#GradientBoostingRegressor Model Classification - (Response_time)
response_time_model = GradientBoostingRegressor(random_state=42)
response_time_model.fit(X_train, y_time_train)
time_pred = response_time_model.predict(X_test)
print("Mean Absolute Error (Response Time):", mean_absolute_error(y_time_test, time_pred))

#LogisticRegression Model Classification - (Outcome)
outcome_model = LogisticRegression(max_iter=10000,solver="lbfgs")
outcome_model.fit(X_train, y_out_train)
outcome_pred = outcome_model.predict(X_test)
print("Outcome Model Accuracy:", accuracy_score(y_out_test, outcome_pred))
print(classification_report(y_out_test, outcome_pred))

#Priority Scoring
def calculate_priority_score(row):
    severity_score = severity_model.predict(pd.DataFrame([row]))[0]
    response_time = response_time_model.predict(pd.DataFrame([row]))[0]
    escalation_risk = outcome_model.predict_proba(pd.DataFrame([row]))[0][1]
    priority_score = (
        0.45 * severity_score +
        0.30 * (response_time / 30) +
        0.15 * (1 - row["ambulance_available"] / 5) +
        0.10 * escalation_risk
    )
    return round(priority_score * 100, 2)

sample_case = {
    "emergency_type": label_encoders["emergency_type"].transform(["heart_attack"])[0],
    "severity_reported": 5,
    "distance_km": 4.0,
    "traffic_level": 4,
    "time_of_day": label_encoders["time_of_day"].transform(["rush_hour"])[0],
    "people_affected": 1,
    "ambulance_available": 1,
    "hospital_icu_available": 1,
    "weather_condition": label_encoders["weather_condition"].transform(["rain"])[0],
    "area_risk_score": 8
}

priority = calculate_priority_score(sample_case)
print("Final Priority Score:", priority)

Severity Model Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        87
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00        37
           3       1.00      1.00      1.00        40

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Mean Absolute Error (Response Time): 1.6253586117419427
Outcome Model Accuracy: 0.795
              precision    recall  f1-score   support

           0       0.79      0.77      0.78        91
           1       0.75      0.82      0.78        68
           2       0.92      0.80      0.86        41

    accuracy                           0.80       200
   macro avg       0.82      0.80      0.81       200
weighted avg       0.80      0.80      0.80       200

Final Priority Score: 62.45


# Rapid-Aid Full Backend

In [10]:
# =========================
# FULL BACKEND IN ONE CELL
# =========================
# 1. Imports & Warning Control
import pandas as pd
import numpy as np
import time
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

# 2. Load Dataset
df = pd.read_csv("emergency_ml_dataset_1000.csv")

# 3. Encode Categorical Data
categorical_cols = [
    "emergency_type",
    "time_of_day",
    "weather_condition",
    "severity_class",
    "outcome"
]

encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# 4. Feature & Label Selection
features = [
    "emergency_type",
    "severity_reported",
    "distance_km",
    "traffic_level",
    "time_of_day",
    "people_affected",
    "ambulance_available",
    "hospital_icu_available",
    "weather_condition",
    "area_risk_score"
]

X = df[features]
y_sev = df["severity_class"]
y_time = df["actual_response_time"]
y_out = df["outcome"]

# 5. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 6. Train-Test Split
X_train, X_test, y_sev_train, y_sev_test = train_test_split(X_scaled, y_sev, test_size=0.2, random_state=42)

# 7. Train ML Models
severity_model = RandomForestClassifier(n_estimators=100, random_state=42)
severity_model.fit(X_train, y_sev_train)

time_model = GradientBoostingRegressor(random_state=42)

# Train-test split for response time model
_, _, y_time_train, y_time_test = train_test_split(
    X_scaled, y_time, test_size=0.2, random_state=42
)

time_model = GradientBoostingRegressor(random_state=42)
time_model.fit(X_train, y_time_train)

# Train-test split for outcome (escalation) model
_, _, y_out_train, y_out_test = train_test_split(
    X_scaled, y_out, test_size=0.2, random_state=42
)
outcome_model = LogisticRegression(max_iter=1000)
outcome_model.fit(X_train, y_out_train)

print("✅ ML Models Trained Successfully")

# 8. In-Memory Backend Databases
AMBULANCES = [
    {"id": "AMB1", "status": "free", "distance": 3},
    {"id": "AMB2", "status": "free", "distance": 6},
    {"id": "AMB3", "status": "busy", "distance": 2}
]

HOSPITALS = [
    {"id": "HOSP1", "icu": True, "distance": 5},
    {"id": "HOSP2", "icu": False, "distance": 3},
    {"id": "HOSP3", "icu": True, "distance": 8}
]

ACTIVE_EMERGENCIES = []

# 9. Backend Functions
def create_emergency(data):
    ACTIVE_EMERGENCIES.append(data)
    print("\n🚨 Emergency Registered")

def calculate_priority(data):
    encoded = [
        encoders["emergency_type"].transform([data["emergency_type"]])[0],
        data["severity_reported"],
        data["distance_km"],
        data["traffic_level"],
        encoders["time_of_day"].transform([data["time_of_day"]])[0],
        data["people_affected"],
        data["ambulance_available"],
        data["hospital_icu_available"],
        encoders["weather_condition"].transform([data["weather_condition"]])[0],
        data["area_risk_score"]
    ]

    X_input = scaler.transform([encoded])

    severity = severity_model.predict(X_input)[0]
    response_time = time_model.predict(X_input)[0]
    escalation = outcome_model.predict_proba(X_input)[0][1]

    priority_score = (
        0.45 * severity +
        0.30 * (response_time / 30) +
        0.15 * (1 - data["ambulance_available"] / 5) +
        0.10 * escalation
    )

    return round(priority_score * 100, 2)

def allocate_resources():
    free_amb = sorted([a for a in AMBULANCES if a["status"] == "free"], key=lambda x: x["distance"])
    icu_hosp = sorted([h for h in HOSPITALS if h["icu"]], key=lambda x: x["distance"])

    primary = free_amb[0] if free_amb else None
    backup = free_amb[1] if len(free_amb) > 1 else None
    hospital = icu_hosp[0] if icu_hosp else None

    return primary, backup, hospital

def reassurance_messages():
    messages = [
        "🚑 Ambulance is on the way. Please stay calm.",
        "⏳ We are just a few minutes away.",
        "❤️ Help is approaching. Stay strong."
    ]
    for msg in messages:
        print(msg)
        time.sleep(1)

def trigger_backup():
    print("⚠️ Delay detected — Backup ambulance activated")

# 10. END-TO-END DEMO RUN
sample_emergency = {
    "emergency_type": "heart_attack",
    "severity_reported": 5,
    "distance_km": 4,
    "traffic_level": 4,
    "time_of_day": "rush_hour",
    "people_affected": 1,
    "ambulance_available": 1,
    "hospital_icu_available": 1,
    "weather_condition": "rain",
    "area_risk_score": 8
}

create_emergency(sample_emergency)

priority = calculate_priority(sample_emergency)
print("🔥 Priority Score:", priority)

primary, backup, hospital = allocate_resources()
print("🚑 Primary Ambulance:", primary)
print("🚑 Backup Ambulance:", backup)
print("🏥 Assigned Hospital:", hospital)

reassurance_messages()
trigger_backup()

print("\n✅ Backend Simulation Completed Successfully")

✅ ML Models Trained Successfully

🚨 Emergency Registered
🔥 Priority Score: 34.35
🚑 Primary Ambulance: {'id': 'AMB1', 'status': 'free', 'distance': 3}
🚑 Backup Ambulance: {'id': 'AMB2', 'status': 'free', 'distance': 6}
🏥 Assigned Hospital: {'id': 'HOSP1', 'icu': True, 'distance': 5}
🚑 Ambulance is on the way. Please stay calm.
⏳ We are just a few minutes away.
❤️ Help is approaching. Stay strong.
⚠️ Delay detected — Backup ambulance activated

✅ Backend Simulation Completed Successfully


In [6]:
# ==========================================================
# COMPLETE EMERGENCY AI SYSTEM (ONE CELL – FINAL)
# ==========================================================

import os, time, warnings
import whisper, spacy, folium
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

# =========================
# 1️⃣ LOAD DATA & ML MODELS
# =========================

df = pd.read_csv("emergency_ml_dataset_1000.csv")

categorical_cols = ["emergency_type","time_of_day","weather_condition","severity_class","outcome"]
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

features = [
    "emergency_type","severity_reported","distance_km","traffic_level",
    "time_of_day","people_affected","ambulance_available",
    "hospital_icu_available","weather_condition","area_risk_score"
]

X = df[features]
y_sev = df["severity_class"]
y_time = df["actual_response_time"]
y_out = df["outcome"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, _, y_sev_train, _ = train_test_split(X_scaled, y_sev, test_size=0.2, random_state=42)
_, _, y_time_train, _ = train_test_split(X_scaled, y_time, test_size=0.2, random_state=42)
_, _, y_out_train, _ = train_test_split(X_scaled, y_out, test_size=0.2, random_state=42)

severity_model = RandomForestClassifier(n_estimators=100)
severity_model.fit(X_train, y_sev_train)

time_model = GradientBoostingRegressor()
time_model.fit(X_train, y_time_train)

outcome_model = LogisticRegression(max_iter=1000)
outcome_model.fit(X_train, y_out_train)

print("✅ ML Models Loaded")

# =========================
# 2️⃣ LOAD NLP MODELS
# =========================

asr_model = whisper.load_model("base")     # Multilingual ASR
nlp = spacy.load("en_core_web_sm")

# =========================
# 3️⃣ GEO & MAP DATA
# =========================

geolocator = Nominatim(user_agent="emergency_ai_system")

AMBULANCES = {
    "AMB1": (13.0800, 80.2600),
    "AMB2": (13.0900, 80.2800)
}

HOSPITALS = {
    "HOSP1": (13.0850, 80.2750)
}

# =========================
# 4️⃣ AUDIO / TEXT INPUT (SAFE)
# =========================

def audio_or_text_input(audio_path=None, manual_text=None):
    if audio_path and os.path.exists(audio_path):
        result = asr_model.transcribe(audio_path)
        return result["text"], result["language"]
    elif manual_text:
        return manual_text, "manual"
    else:
        raise ValueError("No audio or manual text provided")

# =========================
# 5️⃣ NLP + LOCATION EXTRACTION
# =========================

def extract_nlp_features(text):
    doc = nlp(text.lower())

    emergency_type, severity, confidence = "minor_injury", 2, 0.4
    keywords = {
        "heart_attack": ["heart", "chest pain", "breathing"],
        "fire_school": ["fire", "smoke"],
        "road_accident": ["accident", "crash"],
        "gas_leak": ["gas", "leak"]
    }

    for k, words in keywords.items():
        for w in words:
            if w in text.lower():
                emergency_type = k
                severity = 5 if k in ["heart_attack","fire_school"] else 3
                confidence += 0.25

    traffic = 4 if any(t.text in ["traffic","jam"] for t in doc) else 2
    people = 10 if any(t.text in ["children","crowd","many"] for t in doc) else 1

    locations = [ent.text for ent in doc.ents if ent.label_ in ["GPE","LOC"]]
    location = locations[0] if locations else "Chennai"

    return {
        "emergency_type": emergency_type,
        "severity_reported": severity,
        "traffic_level": traffic,
        "time_of_day": "rush_hour" if traffic >= 4 else "normal",
        "people_affected": people,
        "location": location,
        "confidence": min(confidence,1.0)
    }

# =========================
# 6️⃣ PRIORITY SCORING
# =========================

def calculate_priority(data):
    encoded = [
        encoders["emergency_type"].transform([data["emergency_type"]])[0],
        data["severity_reported"], data["distance_km"], data["traffic_level"],
        encoders["time_of_day"].transform([data["time_of_day"]])[0],
        data["people_affected"], data["ambulance_available"],
        data["hospital_icu_available"],
        encoders["weather_condition"].transform([data["weather_condition"]])[0],
        data["area_risk_score"]
    ]

    X_input = scaler.transform([encoded])
    sev = severity_model.predict(X_input)[0]
    resp = time_model.predict(X_input)[0]
    risk = outcome_model.predict_proba(X_input)[0][1]

    return round((0.45*sev + 0.30*(resp/30) + 0.15*(1-data["ambulance_available"]/5) + 0.10*risk)*100,2)

# =========================
# 7️⃣ LIVE MAP PLOTTING
# =========================

def get_coords(place):
    try:
        loc = geolocator.geocode(place)
        if loc:
            return loc.latitude, loc.longitude
    except:
        pass
    return 13.0827, 80.2707  # Chennai fallback

def render_map(location):
    lat, lon = get_coords(location)
    m = folium.Map(location=[lat, lon], zoom_start=13)

    folium.Marker([lat, lon], popup="🚨 Emergency", icon=folium.Icon(color="red")).add_to(m)

    for amb, (a,b) in AMBULANCES.items():
        folium.Marker([a,b], popup=f"🚑 {amb}", icon=folium.Icon(color="blue")).add_to(m)
        folium.PolyLine([[a,b],[lat,lon]], color="blue").add_to(m)

    for hosp, (a,b) in HOSPITALS.items():
        folium.Marker([a,b], popup="🏥 Hospital", icon=folium.Icon(color="purple")).add_to(m)

    return m

# =========================
# 8️⃣ FULL PIPELINE
# =========================

def process_emergency(audio_path=None, manual_text=None):
    text, mode = audio_or_text_input(audio_path, manual_text)
    info = extract_nlp_features(text)

    if info["confidence"] < 0.6:
        print("⚠️ OPERATOR ALERT: Low confidence input – verify details")

    enriched = {
        **info,
        "distance_km": 4.0,
        "ambulance_available": 1,
        "hospital_icu_available": 1,
        "weather_condition": "rain",
        "area_risk_score": 8
    }

    priority = calculate_priority(enriched)

    print("\n📞 INPUT TEXT:\n", text)
    print("🧾 Input Mode:", mode)
    print("📍 Location:", info["location"])
    print("🔥 Priority Score:", priority)

    return render_map(info["location"])

# =========================
# 9️⃣ DEMO RUN (SAFE MODE)
# =========================

process_emergency(
    manual_text="""
    My father has severe chest pain near Anna Nagar.
    Traffic is very heavy. Please send an ambulance immediately.
    """
)

✅ ML Models Loaded

📞 INPUT TEXT:
 
    My father has severe chest pain near Anna Nagar.
    Traffic is very heavy. Please send an ambulance immediately.
    
🧾 Input Mode: manual
📍 Location: Chennai
🔥 Priority Score: 34.35


In [24]:
# ==========================================================
# LIVE DEMO: AUDIO → NLP → ML → OPERATOR OUTPUT
# ==========================================================

import os, warnings
import whisper, spacy
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

# ----------------------------------------------------------
# OPTIONAL MAP (SAFE IMPORT)
# ----------------------------------------------------------
try:
    import folium
    from geopy.geocoders import Nominatim
    FOLIUM_AVAILABLE = True
except:
    FOLIUM_AVAILABLE = False

# ==========================================================
# 1️⃣ LOAD DATA & TRAIN MODELS (FAST)
# ==========================================================

df = pd.read_csv("emergency_ml_dataset_1000.csv")

cat_cols = ["emergency_type","time_of_day","weather_condition","severity_class","outcome"]
encoders = {}
for c in cat_cols:
    le = LabelEncoder()
    df[c] = le.fit_transform(df[c])
    encoders[c] = le

X = df[
    ["emergency_type","severity_reported","distance_km","traffic_level",
     "time_of_day","people_affected","ambulance_available",
     "hospital_icu_available","weather_condition","area_risk_score"]
]

y_sev = df["severity_class"]
y_time = df["actual_response_time"]
y_out = df["outcome"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, _, y_sev_train, _ = train_test_split(X_scaled, y_sev, test_size=0.2)
_, _, y_time_train, _ = train_test_split(X_scaled, y_time, test_size=0.2)
_, _, y_out_train, _ = train_test_split(X_scaled, y_out, test_size=0.2)

sev_model = RandomForestClassifier(n_estimators=80)
sev_model.fit(X_train, y_sev_train)

time_model = GradientBoostingRegressor()
time_model.fit(X_train, y_time_train)

out_model = LogisticRegression(max_iter=800)
out_model.fit(X_train, y_out_train)

print("✅ ML Models Ready")

# ==========================================================
# 2️⃣ LOAD WHISPER + NLP
# ==========================================================

asr = whisper.load_model("base")
nlp = spacy.load("en_core_web_sm")

# ==========================================================
# 3️⃣ AUDIO → TEXT
# ==========================================================

import soundfile as sf
import numpy as np

def transcribe_audio(file_path):
    # Load WAV manually (NO ffmpeg)
    audio, sr = sf.read(file_path)

    # Ensure mono
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)

    # Whisper expects float32
    audio = audio.astype(np.float32)

    result = asr.transcribe(audio)
    return result["text"], result["language"]


# ==========================================================
# 4️⃣ NLP EXTRACTION
# ==========================================================

def extract_call_details(text):
    doc = nlp(text.lower())

    emergency_type = "minor_injury"
    severity = 2
    confidence = 0.4

    if "heart" in text or "chest pain" in text:
        emergency_type = "heart_attack"
        severity = 5
        confidence += 0.3
    elif "fire" in text:
        emergency_type = "fire_school"
        severity = 5
        confidence += 0.3
    elif "accident" in text:
        emergency_type = "road_accident"
        severity = 3
        confidence += 0.2

    traffic = 4 if "traffic" in text or "jam" in text else 2
    people = 1 if "my father" in text or "one person" in text else 5

    locs = [ent.text for ent in doc.ents if ent.label_ in ["GPE","LOC"]]
    location = locs[0] if locs else "Chennai"

    return {
        "emergency_type": emergency_type,
        "severity_reported": severity,
        "traffic_level": traffic,
        "time_of_day": "rush_hour" if traffic >= 4 else "normal",
        "people_affected": people,
        "location": location,
        "confidence": min(confidence, 1.0)
    }

# ==========================================================
# 5️⃣ PRIORITY SCORING
# ==========================================================

def calculate_priority(data):
    encoded = [
        encoders["emergency_type"].transform([data["emergency_type"]])[0],
        data["severity_reported"], 4.0, data["traffic_level"],
        encoders["time_of_day"].transform([data["time_of_day"]])[0],
        data["people_affected"], 1, 1,
        encoders["weather_condition"].transform(["rain"])[0],
        8
    ]

    X_input = scaler.transform([encoded])
    s = sev_model.predict(X_input)[0]
    t = time_model.predict(X_input)[0]
    r = out_model.predict_proba(X_input)[0][1]

    return round((0.4*s + 0.3*(t/30) + 0.2*0.5 + 0.1*r)*100, 2)

# ==========================================================
# 6️⃣ MAP RENDER (OPTIONAL)
# ==========================================================

def render_map(location):
    if not FOLIUM_AVAILABLE:
        print("🗺️ Map unavailable — location:", location)
        return None

    geo = Nominatim(user_agent="demo")
    loc = geo.geocode(location)
    lat, lon = (loc.latitude, loc.longitude) if loc else (13.08, 80.27)

    m = folium.Map(location=[lat, lon], zoom_start=13)
    folium.Marker([lat, lon], popup="🚨 Emergency").add_to(m)
    return m

# ==========================================================
# 7️⃣ RUN LIVE DEMO
# ==========================================================

print("\n🎧 Processing Emergency Call Audio...\n")

text, lang = transcribe_audio(r"C:\Users\Admin\Desktop\AIML\Rapid-Aid\Audio.wav")
details = extract_call_details(text)
priority = calculate_priority(details)

print("📞 Transcribed Call:")
print(text)

print("\n📍 Location Detected:", details["location"])
print("🧠 Confidence Score:", details["confidence"])

if details["confidence"] < 0.6:
    print("⚠️ OPERATOR ALERT: Verify caller details manually")

print("\n🔥 FINAL AI PRIORITY SCORE:", priority)

render_map(details["location"])

✅ ML Models Ready

🎧 Processing Emergency Call Audio...

📞 Transcribed Call:
 I wish you a happy life! Complete

📍 Location Detected: Chennai
🧠 Confidence Score: 0.4
⚠️ OPERATOR ALERT: Verify caller details manually

🔥 FINAL AI PRIORITY SCORE: 162.05


In [23]:
import sys
!{sys.executable} -m pip install soundfile

  Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl.metadata (16 kB)
Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl (1.0 MB)


In [22]:
!pip install soundfile

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 575.9 kB/s eta 0:00:01
   -------------------- ------------------- 0.5/1.0 MB 575.9 kB/s eta 0:00:01
   ------------------------------ --------- 0.8/1.0 MB 673.2 kB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 735.7 kB/s  0:00:01
